In [ ]:
%%writefile b.cpp

//컴파일 g++ -O3 -pthread Thread_test_2w.cpp -o Thread_test_2w.exe 
//실행 ./Thread_test_2w.exe

#include <math.h>
#include <cstdio>
#include <iostream>
#include <thread>
#include <vector>
#include <chrono>

using std::thread;
using std::vector;


int threadIdx(long nds, long *distIdx){ 
    int i, threads, rest;   
    long range1; //1개 스레드 커버 범위
    int rTreads=0; //유효 스레드수

    threads =17;   //cpu core 8개 가정
    range1 =nds/threads; //range1 변수가 int라서 몫만 전달
    rest =nds %threads;

    distIdx[0]=0;
    for(i=0;i<threads;i++) distIdx[i+1] =(i < rest ? range1 +1 : range1 );    //스레드 계산수 차를 1로 최소화    
    for(i=1;i<threads+1;i++) if(distIdx[i]> 0) rTreads++;                   //유용 스레드 수가 노드 수보다 많을 경우 유효스레드 수 취하기
    for(i=1;i<threads+1;i++) if(distIdx[i] >0) distIdx[i] =distIdx[i-1] +distIdx[i];    //스레드별 for index 만들기

    return rTreads;
}

//******************************************************************* //
void func_thread_worker(long ndSt, long ndEd,float *rstV ) 
{
    for(int i=ndSt; i<ndEd; i++) rstV[i]= exp(ndSt);
}

void func_thread(int outns) 
{
    long i, N, C, H, Nn, Cn, Hn, nn, rst_cnt;
    float *rstS, *rstV;

    int trds;
    long trdIndx[101];

    Nn =1; Cn =100; Hn =100; nn =outns;
 
    rstS = (float *)malloc(sizeof(float)*Nn*Cn*Hn*nn); //in의 R, outNode의 컬럼
    rstV = (float *)malloc(sizeof(float)*nn);        

    trds =threadIdx(nn, trdIndx);
    vector<thread> workers(trds);

    rst_cnt=0;
    for(N=0; N<Nn;N++) for(C=0; C<Cn;C++) for(H=0; H<Hn;H++){ 

        for (i=0; i<trds; i++) workers[i] =thread(func_thread_worker, trdIndx[i], trdIndx[i+1], rstV);
        for (i=0; i<trds; i++) workers[i].join();

        for(i=0; i<nn; i++){
            *(rstS +rst_cnt) =rstV[i];
            rst_cnt++;
        }                     
    }  
}

void func_normal(int outns) //, layer_stat& ls)
{
    long i, N, C, H, Nn, Cn, Hn, nn, rst_cnt;
    float *rstS, *rstV;

    int trds;
    long trdIndx[101];

    Nn =1; Cn =100; Hn =100; nn =outns;
 
    rstS = (float *)malloc(sizeof(float)*Nn*Cn*Hn*nn); //in의 R, outNode의 컬럼
    rstV = (float *)malloc(sizeof(float)*nn);        

    rst_cnt=0;
    for(N=0; N<Nn;N++) for(C=0; C<Cn;C++) for(H=0; H<Hn;H++){ 

        for(i=0; i<nn; i++) rstV[i]= exp(i);

        for(i=0; i<nn; i++){
            *(rstS +rst_cnt) =rstV[i];
            rst_cnt++;
        }                     
    }
}

int main()
{
    
    int rstLen;

    auto sTime = std::chrono::high_resolution_clock::now();

    func_thread(5000); //<<=====
    auto eTime = std::chrono::high_resolution_clock::now(); 
    std::chrono::duration<double> gapTime = eTime - sTime;
    printf("Thread Result Time =%f \n",gapTime.count());

    
    sTime = std::chrono::high_resolution_clock::now();  

    func_normal(5000); //<<=====
    eTime = std::chrono::high_resolution_clock::now(); 
    gapTime = eTime - sTime;
    printf("Normal Time =%f \n ",gapTime.count());


    return 0;
}

In [ ]:
%%shell
g++ -pthread b.cpp -o b.exe
./b.exe